In [43]:
# 1️⃣ Hospital & department capacity setup (UKE-focused)

import random

# --- ICU units (real counts) -----------------------------------------------
# Each ICU must reserve 1 bed for resuscitation => operational_beds = total_beds - 1
ICU_TOTAL_BEDS = {
    "neurochirurgical_icu": 12,
    "neurological_icu": 12,
    "interdis_stage1": 12,     # UKE 1C
    "interdis_stage2": 12,     # UKE 1D
    "interdis_stage3": 12,     # UKE 1E
    "surgical_icu": 12,        # UKE 1F
    "internal_medicine_icu": 12,  # UKE 1G
    "cardio_icu": 12,          # UKE H1b (cardiology)
    "cardio_surgery_icu": 12,  # UKE H1b (cardiac surgery)
    "vascular_cardiac_icu": 8  # UKE H2b
}

ICU_OPERATIONAL_BEDS = {
    k: max(v - 1, 0) for k, v in ICU_TOTAL_BEDS.items()
}

# --- Non-ICU departments (simulated sizes) ----------------------------------
# Rule of thumb: Large=50, Medium=30, Small=20
DEPARTMENTS_SIZE = {
    # Large
    "internal_medicine": 50,
    "neurology": 50,
    "surgery_general": 50,

    # Medium
    "cardiology": 30,
    "pulmonology": 30,
    "gastroenterology": 30,
    "orthopedics": 30,
    "surgery_trauma": 30,
    "vascular_surgery": 30,

    # Small
    "oncology": 20,
    "pediatrics": 20,
    "obstetrics_gynecology": 20,
    "psychiatry": 20,
    "palliative": 20,
    "imc_internal_medicine": 20,
}

# Combine into a single capacity registry (operational bed counts)
OPER_BEDS = {
    **ICU_OPERATIONAL_BEDS,
    **DEPARTMENTS_SIZE
}

# --- Availability sampling ---------------------------------------------------
# Availability is sampled per patient/event: 0%..50% free, with a small chance of 0% hard block
def sample_availability_fraction(hard_block_prob: float = 0.10) -> float:
    """Return a fraction of free capacity in [0.0, 0.5]. Sometimes forced to 0.0."""
    if random.random() < hard_block_prob:
        return 0.0
    return random.uniform(0.0, 0.5)

def capacity_snapshot_normalized(hard_block_prob: float = 0.10) -> dict:
    """
    Produce a dict of normalized capacity scores per department (0..1),
    computed as available_beds / operational_beds for the current moment.
    """
    snap = {}
    for dept, oper_beds in OPER_BEDS.items():
        if oper_beds <= 0:
            snap[dept] = 0.0
            continue
        frac = sample_availability_fraction(hard_block_prob)
        available = int(round(oper_beds * frac))
        snap[dept] = available / oper_beds
    return snap

# Convenience lists if needed later
ICU_DEPARTMENTS = list(ICU_OPERATIONAL_BEDS.keys())
NON_ICU_DEPARTMENTS = list(DEPARTMENTS_SIZE.keys())
ALL_DEPARTMENTS = list(OPER_BEDS.keys())

# Sanity print (optional)
print("ICU operational beds:", ICU_OPERATIONAL_BEDS)
print("Non-ICU sizes:", DEPARTMENTS_SIZE)


ICU operational beds: {'neurochirurgical_icu': 11, 'neurological_icu': 11, 'interdis_stage1': 11, 'interdis_stage2': 11, 'interdis_stage3': 11, 'surgical_icu': 11, 'internal_medicine_icu': 11, 'cardio_icu': 11, 'cardio_surgery_icu': 11, 'vascular_cardiac_icu': 7}
Non-ICU sizes: {'internal_medicine': 50, 'neurology': 50, 'surgery_general': 50, 'cardiology': 30, 'pulmonology': 30, 'gastroenterology': 30, 'orthopedics': 30, 'surgery_trauma': 30, 'vascular_surgery': 30, 'oncology': 20, 'pediatrics': 20, 'obstetrics_gynecology': 20, 'psychiatry': 20, 'palliative': 20, 'imc_internal_medicine': 20}


In [44]:
# 2️⃣ Patient generator using real ICU and simulated non-ICU capacities

import uuid
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Fixed suspected conditions for now
SUSPECTED_CONDITIONS = [
    "cardiac", "respiratory", "neurological",
    "gastrointestinal", "oncology", "infection", "trauma"
]

HOME_TYPES = ["private", "nursing_home"]

def generate_patient():
    # Static patient attributes
    patient_id = str(uuid.uuid4())[:8]
    age = np.random.randint(18, 95)
    home_type = np.random.choice(HOME_TYPES)
    suspected_condition = np.random.choice(SUSPECTED_CONDITIONS)
    
    # EMS and ED timing
    ems_start_time = datetime(2025, 8, 7) + timedelta(
        hours=np.random.randint(0, 24),
        minutes=np.random.randint(0, 60)
    )
    ems_triage_code = np.random.randint(1, 5)  # 1=most urgent, 4=least
    ems_target_hospital = "UKE"
    ed_arrival_time = ems_start_time + timedelta(minutes=np.random.randint(5, 30))

    # Capacity snapshot for all departments (normalized)
    capacities = capacity_snapshot_normalized()

    # Target: for now, simple suitability score for ward assignment
    ward_assignment_suitability = np.random.uniform(0, 1)

    # Combine all fields
    record = {
        "patient_id": patient_id,
        "age": age,
        "home_type": home_type,
        "suspected_condition": suspected_condition,
        "ems_start_time": ems_start_time,
        "ems_triage_code": ems_triage_code,
        "ems_target_hospital": ems_target_hospital,
        "ed_arrival_time": ed_arrival_time,
    }

    # Add department capacities as features
    for dept, cap in capacities.items():
        record[f"cap_{dept}"] = cap

    # Output target(s)
    record["ward_assignment_suitability"] = ward_assignment_suitability

    return record

def generate_dataset(n_patients=500):
    patients = [generate_patient() for _ in range(n_patients)]
    return pd.DataFrame(patients)

# Quick test
df = generate_dataset(5)
print(df.head())
print("\nShape:", df.shape)


  patient_id  age     home_type suspected_condition      ems_start_time  \
0   7bf4c439   70  nursing_home             cardiac 2025-08-07 05:47:00   
1   759d057a   48  nursing_home        neurological 2025-08-07 11:04:00   
2   31d6dcaf   88  nursing_home         respiratory 2025-08-07 06:22:00   
3   cac004ec   22       private           infection 2025-08-07 03:01:00   
4   f39411b9   29       private         respiratory 2025-08-07 22:25:00   

   ems_triage_code ems_target_hospital     ed_arrival_time  \
0                3                 UKE 2025-08-07 06:09:00   
1                2                 UKE 2025-08-07 11:30:00   
2                3                 UKE 2025-08-07 06:47:00   
3                4                 UKE 2025-08-07 03:24:00   
4                2                 UKE 2025-08-07 22:53:00   

   cap_neurochirurgical_icu  cap_neurological_icu  ...  cap_orthopedics  \
0                  0.090909              0.363636  ...         0.300000   
1                  0.09090

In [45]:
# Cell “3A” — ICU mapping + label functions
# ICU preference by condition (simple, editable)
ICU_PREFS = {
    "cardiac": ["cap_cardio_icu", "cap_cardio_surgery_icu", "cap_vascular_cardiac_icu"],
    "respiratory": ["cap_internal_medicine_icu", "cap_interdis_stage1", "cap_interdis_stage2", "cap_interdis_stage3"],
    "neurological": ["cap_neurological_icu", "cap_neurochirurgical_icu", "cap_interdis_stage1"],
    "infection": ["cap_internal_medicine_icu", "cap_interdis_stage2"],
    "trauma": ["cap_surgical_icu", "cap_interdis_stage3"],
    "oncology": ["cap_internal_medicine_icu", "cap_interdis_stage2"],
    "gastrointestinal": ["cap_surgical_icu", "cap_internal_medicine_icu"]
}

def best_relevant_icu_capacity(rec: dict) -> float:
    """Return the max normalized capacity across relevant ICUs for this condition."""
    cond = rec["suspected_condition"]
    keys = ICU_PREFS.get(cond, [])
    if not keys:
        return 0.0
    vals = [float(rec.get(k, 0.0)) for k in keys]
    return max(vals) if vals else 0.0

def synth_icu_labels(rec: dict) -> dict:
    """
    Make labels depend on ICU capacity + acuity.
    - icu_assignment_suitability: higher if relevant ICU has capacity.
    - icu_bottleneck_risk: high when relevant ICU is near zero.
    - pathway_latency_min: inflates when bottleneck risk is high (ED boarding).
    - route_plan: EMS→ICU if urgent and capacity; else EMS→ED→ICU.
    """
    cap_rel = best_relevant_icu_capacity(rec)            # 0..1
    triage = int(rec["ems_triage_code"])                 # 1 (most urgent) .. 4
    urgent = (triage <= 2)

    # Suitability: base + positive weight on relevant ICU capacity, small noise
    base_by_cond = {
        "cardiac": 0.70, "respiratory": 0.68, "neurological": 0.67,
        "infection": 0.62, "trauma": 0.66, "oncology": 0.60, "gastrointestinal": 0.63
    }
    base = base_by_cond.get(rec["suspected_condition"], 0.62)
    suitability = base + 0.35*cap_rel - (0.05 if urgent and cap_rel < 0.15 else 0.0)
    suitability = float(np.clip(suitability + np.random.normal(0, 0.03), 0, 1))

    # Bottleneck risk: inverse of capacity with urgency emphasis
    bottleneck = 1.0 - cap_rel
    if urgent:
        bottleneck = np.clip(bottleneck + 0.05, 0, 1)

    # Latency model (minutes): base + penalty for bottleneck
    # Base ~ 60–120 min; add 120–240 min when bottleneck high (boarding)
    base_latency = np.random.uniform(60, 120)
    penalty = np.interp(bottleneck, [0, 1], [0, np.random.uniform(120, 240)])
    pathway_latency_min = float(base_latency + penalty)

    # Route plan decision
    if urgent and cap_rel >= 0.25:
        route_plan = "EMS→ICU"
    else:
        route_plan = "EMS→ED→ICU"

    return {
        "icu_assignment_suitability": suitability,
        "icu_bottleneck_risk": float(np.clip(bottleneck, 0, 1)),
        "pathway_latency_min": pathway_latency_min,
        "route_plan": route_plan
    }


In [48]:
# Cell 3B — Patch generate_patient() to use ICU-driven labels
# --- ICU preferences by suspected condition
ICU_PREFS = {
    "cardiac": ["cap_cardio_icu","cap_cardio_surgery_icu","cap_vascular_cardiac_icu"],
    "respiratory": ["cap_internal_medicine_icu","cap_interdis_stage1","cap_interdis_stage2","cap_interdis_stage3"],
    "neurological": ["cap_neurological_icu","cap_neurochirurgical_icu","cap_interdis_stage1"],
    "infection": ["cap_internal_medicine_icu","cap_interdis_stage2"],
    "trauma": ["cap_surgical_icu","cap_interdis_stage3"],
    "oncology": ["cap_internal_medicine_icu","cap_interdis_stage2"],
    "gastrointestinal": ["cap_surgical_icu","cap_internal_medicine_icu"],
}

def best_relevant_icu_capacity(rec: dict) -> float:
    keys = ICU_PREFS.get(rec["suspected_condition"], [])
    vals = [float(rec.get(k, 0.0)) for k in keys]
    return max(vals) if vals else 0.0

def synth_icu_labels(rec: dict) -> dict:
    cap_rel = best_relevant_icu_capacity(rec)  # 0..1
    triage = int(rec["ems_triage_code"])
    urgent = (triage <= 2)

    # base suitability by condition
    base = {
        "cardiac":0.70,"respiratory":0.68,"neurological":0.67,
        "infection":0.62,"trauma":0.66,"oncology":0.60,"gastrointestinal":0.63
    }.get(rec["suspected_condition"], 0.62)

    suitability = base + 0.35*cap_rel - (0.05 if urgent and cap_rel < 0.15 else 0.0)
    suitability = float(np.clip(suitability + np.random.normal(0,0.03), 0, 1))

    bottleneck = 1.0 - cap_rel
    if urgent:
        bottleneck = np.clip(bottleneck + 0.05, 0, 1)

    base_latency = np.random.uniform(60, 120)  # base in minutes
    penalty = np.interp(bottleneck, [0,1], [0, np.random.uniform(120,240)])
    pathway_latency_min = float(base_latency + penalty)

    route_plan = "EMS→ICU" if (urgent and cap_rel >= 0.25) else "EMS→ED→ICU"

    return {
        "icu_assignment_suitability": suitability,
        "icu_bottleneck_risk": float(bottleneck),
        "pathway_latency_min": pathway_latency_min,
        "route_plan": route_plan
    }

# --- Updated generator with ICU + Non-ICU capacities and ICU labels
def generate_patient():
    patient_id = str(uuid.uuid4())[:8]

    # Demographics
    age = np.random.randint(18, 95)
    home_type = random.choice(['private', 'nursing_home'])

    # Clinical presentation
    suspected_condition = random.choice([
        'cardiac', 'respiratory', 'neurological',
        'infection', 'trauma', 'oncology', 'gastrointestinal'
    ])

    # EMS & ED timing
    ems_start_time = datetime(2025, 8, 7) + timedelta(
        hours=np.random.randint(0, 24),
        minutes=np.random.randint(0, 60)
    )
    ems_triage_code = np.random.choice([1, 2, 3, 4], p=[0.15, 0.35, 0.35, 0.15])
    ems_target_hospital = "UKE"
    ed_arrival_time = ems_start_time + timedelta(minutes=np.random.randint(5, 25))

    day_of_week = ed_arrival_time.strftime("%A")
    hour_of_day = ed_arrival_time.hour

    # ICU capacities (n-1 beds operational)
    icu_capacities = {
        'neurochirurgical_icu': 11,
        'neurological_icu': 11,
        'interdis_stage1': 11,
        'interdis_stage2': 11,
        'interdis_stage3': 11,
        'surgical_icu': 11,
        'internal_medicine_icu': 11,
        'cardio_icu': 11,
        'cardio_surgery_icu': 11,
        'vascular_cardiac_icu': 7
    }
    icu_caps = {f"cap_{k}": round(np.random.randint(0, beds + 1) / beds, 2)
                for k, beds in icu_capacities.items()}

    # Non-ICU capacities (0–50% availability)
    non_icu_sizes = {
        'internal_medicine': 50, 'neurology': 50, 'surgery_general': 50,
        'cardiology': 30, 'pulmonology': 30, 'gastroenterology': 30,
        'orthopedics': 30, 'surgery_trauma': 30, 'vascular_surgery': 30,
        'oncology': 20, 'pediatrics': 20, 'obstetrics_gynecology': 20,
        'psychiatry': 20, 'palliative': 20, 'imc_internal_medicine': 20
    }
    non_icu_caps = {f"cap_{k}": round(np.random.randint(0, int(beds*0.5) + 1) / beds, 2)
                    for k, beds in non_icu_sizes.items()}

    # Assemble record
    record = {
        "patient_id": patient_id,
        "age": age,
        "home_type": home_type,
        "suspected_condition": suspected_condition,
        "ems_start_time": ems_start_time,
        "ems_triage_code": int(ems_triage_code),
        "ems_target_hospital": ems_target_hospital,
        "ed_arrival_time": ed_arrival_time,
        "day_of_week": day_of_week,
        "hour_of_day": hour_of_day,
        **icu_caps,
        **non_icu_caps,
    }

    # ICU-driven labels
    record.update(synth_icu_labels(record))
    return record





In [49]:
# Cell 4 — Regenerate dataset (now includes ICU-driven labels)

# Rebuild a fresh dataset with ICU-driven labels
df = pd.DataFrame(generate_dataset(n_patients=500))

# quick hygiene
assert df.isna().sum().sum() == 0
assert (df["ems_triage_code"].between(1,4)).all()
print(df.shape, "cols:", len(df.columns))
display(df.head(2)[[
    "suspected_condition","ems_triage_code","icu_assignment_suitability",
    "icu_bottleneck_risk","pathway_latency_min","route_plan"
]])


(500, 39) cols: 39


,suspected_condition,ems_triage_code,icu_assignment_suitability,icu_bottleneck_risk,pathway_latency_min,route_plan
0,gastrointestinal,4,0.627866,0.82,253.228274,EMS→ED→ICU
1,infection,2,0.909965,0.23,110.049853,EMS→ICU


In [ ]:
# Cell 5 — Feature lists & split (no sex/arrival_mode)

from sklearn.model_selection import train_test_split

TARGET = "icu_assignment_suitability"

FEATS_CAT = ["home_type", "suspected_condition", "day_of_week"]
CAP_COLS = [c for c in df.columns if c.startswith("cap_")]  # ICU + non-ICU capacity snapshot
FEATS_NUM = ["age", "ems_triage_code", "hour_of_day"] + CAP_COLS

X = df[FEATS_CAT + FEATS_NUM]
y = df[TARGET].astype("float32")

X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val,  X_test, y_val,  y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=42)

print("Train/Val/Test:", X_train.shape, X_val.shape, X_test.shape)


In [ ]:
# Ridge regression sanity check
import time
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

start_time = time.time()

# Preprocessor (re-using FEATS_CAT and FEATS_NUM from before)
pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), FEATS_CAT),
    ("num", StandardScaler(), FEATS_NUM)
])

model = Pipeline([
    ("pre", pre),
    ("reg", Ridge(alpha=1.0))
])

# Train
model.fit(X_train, y_train)

# Validate
y_val_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_val_pred)
mae = mean_absolute_error(y_val, y_val_pred)

end_time = time.time()

print(f"Validation MSE: {mse:.4f}")
print(f"Validation MAE: {mae:.4f}")
print(f"Training time: {end_time - start_time:.2f} seconds")


In [ ]:
# 🔹 Step 1: Preprocess with expanded feature set
FEATS_CAT = ["home_type", "suspected_condition", "day_of_week"]
FEATS_NUM = [
    "age", "ems_triage_code", "hour_of_day",
    # All capacity columns
] + [col for col in df.columns if col.startswith("cap_")]

TARGET = "ward_assignment_suitability"

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), FEATS_CAT),
    ("num", StandardScaler(), FEATS_NUM)
])

X_train_np = pre.fit_transform(X_train)
X_val_np   = pre.transform(X_val)
X_test_np  = pre.transform(X_test)

y_train_np = y_train.values.astype("float32")
y_val_np   = y_val.values.astype("float32")
y_test_np  = y_test.values.astype("float32")

print("Shapes:", X_train_np.shape, X_val_np.shape, X_test_np.shape)

# 🔹 Step 2: Convert to torch tensors and loaders
import torch
from torch.utils.data import TensorDataset, DataLoader

Xtr = torch.tensor(X_train_np, dtype=torch.float32)
ytr = torch.tensor(y_train_np.reshape(-1, 1), dtype=torch.float32)
Xva = torch.tensor(X_val_np, dtype=torch.float32)
yva = torch.tensor(y_val_np.reshape(-1, 1), dtype=torch.float32)
Xte = torch.tensor(X_test_np, dtype=torch.float32)
yte = torch.tensor(y_test_np.reshape(-1, 1), dtype=torch.float32)

train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=64, shuffle=True)
val_loader   = DataLoader(TensorDataset(Xva, yva), batch_size=256)
test_loader  = DataLoader(TensorDataset(Xte, yte), batch_size=256)

input_dim = Xtr.shape[1]
print("Input dim:", input_dim)

# 🔹 Step 3: Define the MLP
import torch.nn as nn
import torch.optim as optim
import numpy as np

class MLP(nn.Module):
    def __init__(self, d_in):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, 128), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x)

mlp = MLP(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(mlp.parameters(), lr=1e-3)

# 🔹 Step 4: Training loop with early stopping
best_val = np.inf
patience, bad = 8, 0
epochs = 50

for ep in range(1, epochs + 1):
    mlp.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred = mlp(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader.dataset)

    mlp.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            val_loss += criterion(mlp(xb), yb).item() * xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {ep:02d} | train MSE {train_loss:.4f} | val MSE {val_loss:.4f}")

    if val_loss + 1e-6 < best_val:
        best_val = val_loss
        bad = 0
        best_state = {k: v.cpu().clone() for k, v in mlp.state_dict().items()}
    else:
        bad += 1
        if bad >= patience:
            print("Early stopping.")
            break

mlp.load_state_dict(best_state)


In [ ]:
# Test set evaluation
mlp.eval()
with torch.no_grad():
    y_pred_test = np.vstack([mlp(xb).cpu().numpy() for xb, _ in test_loader]).ravel()

mse_test = mean_squared_error(y_test_np, y_pred_test)
mae_test = mean_absolute_error(y_test_np, y_pred_test)

print(f"MLP Test MSE: {mse_test:.4f} | MAE: {mae_test:.4f}")

# Scatter plot
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test_np, y=y_pred_test, alpha=0.7)
plt.plot([0,1], [0,1], 'r--', label="Perfect")
plt.xlabel("Actual suitability")
plt.ylabel("Predicted suitability")
plt.title("MLP — Predicted vs Actual Suitability")
plt.legend()
plt.show()


In [ ]:
# Cell PI-1 — Get feature names + define a predict helper
import numpy as np

# Recover feature names from the fitted preprocessor used for X_*_np
cat_names = pre.named_transformers_["cat"].get_feature_names_out(FEATS_CAT)
num_names = np.array(FEATS_NUM, dtype=str)
feature_names = np.concatenate([cat_names, num_names])

assert feature_names.shape[0] == X_test_np.shape[1], (
    f"Name/feature mismatch: {feature_names.shape[0]} vs {X_test_np.shape[1]}"
)

# Torch → numpy prediction helper on raw numpy features
import torch

def mlp_predict_numpy(X_np: np.ndarray) -> np.ndarray:
    X_tensor = torch.tensor(X_np, dtype=torch.float32)
    mlp.eval()
    with torch.no_grad():
        y_hat = mlp(X_tensor).cpu().numpy().ravel()
    return y_hat



In [ ]:
# Cell PI-2 — Permutation importance (on test set)
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt

def permutation_importance_torch(predict_fn, X, y, n_repeats=10, seed=42):
    rng = np.random.default_rng(seed)
    baseline = mean_squared_error(y, predict_fn(X))
    importances = np.zeros(X.shape[1], dtype=float)

    for j in range(X.shape[1]):
        losses = []
        X_perm = X.copy()
        for _ in range(n_repeats):
            rng.shuffle(X_perm[:, j])   # in-place column shuffle
            y_hat = predict_fn(X_perm)
            losses.append(mean_squared_error(y, y_hat))
        importances[j] = np.mean(losses) - baseline
    return importances, baseline

imps, base_mse = permutation_importance_torch(mlp_predict_numpy, X_test_np, y_test_np, n_repeats=20)

imp_df = pd.DataFrame({
    "feature": feature_names,
    "perm_mse_increase": imps
}).sort_values("perm_mse_increase", ascending=False)

print(f"Baseline test MSE: {base_mse:.4f}")
display(imp_df.head(20))

# Plot top 15
top = imp_df.head(15).iloc[::-1]  # reverse for nicer barh
plt.figure(figsize=(8, 6))
plt.barh(top["feature"], top["perm_mse_increase"])
plt.xlabel("MSE increase when permuted")
plt.title("Permutation Importance (Test set) — Top 15 features")
plt.tight_layout()
plt.show()


In [ ]:
# Cell PI-3 — Quick grouping summary
# This helps see ICU if capacities matter more than non-ICU.

imp_df["group"] = np.where(
    imp_df["feature"].str.contains("cap_"),
    np.where(
        imp_df["feature"].str.contains("cap_(neurochirurgical_icu|neurological_icu|interdis_stage|surgical_icu|internal_medicine_icu|cardio_icu|cardio_surgery_icu|vascular_cardiac_icu)"),
        "ICU capacity",
        "Non-ICU capacity"
    ),
    "Other"
)

group_summary = imp_df.groupby("group")["perm_mse_increase"].sum().reset_index().sort_values("perm_mse_increase", ascending=False)
display(group_summary)


In [ ]:
# 1️⃣ Identify ICU capacity features
icu_features = [
    'cap_neurochirurgical_icu',
    'cap_neurological_icu',
    'cap_interdis_stage1',
    'cap_interdis_stage2',
    'cap_interdis_stage3',
    'cap_surgical_icu',
    'cap_internal_medicine_icu',
    'cap_cardio_icu',
    'cap_cardio_surgery_icu',
    'cap_vascular_cardiac_icu'
]

# 2️⃣ Correlation with target (train split only to avoid test leakage)
correlations = {}
for feat in icu_features:
    correlations[feat] = X_train[feat].corr(y_train)

# Sort by absolute correlation
cor_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Pearson_corr'])
cor_df['abs_corr'] = cor_df['Pearson_corr'].abs()
cor_df.sort_values('abs_corr', ascending=False)


In [ ]:
# 1️⃣ Reduced feature set without ICU
FEATS_CAT_no_icu = [f for f in FEATS_CAT if not f.startswith('cap_')]  # categorical unaffected
FEATS_NUM_no_icu = [f for f in FEATS_NUM if f not in icu_features]      # drop ICU from numeric

# Combine
X_train_no_icu = X_train.drop(columns=icu_features)
X_val_no_icu   = X_val.drop(columns=icu_features)
X_test_no_icu  = X_test.drop(columns=icu_features)

# 2️⃣ Preprocess
pre_no_icu = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), FEATS_CAT),
    ("num", StandardScaler(), FEATS_NUM_no_icu)
])

X_train_np_no_icu = pre_no_icu.fit_transform(X_train_no_icu)
X_val_np_no_icu   = pre_no_icu.transform(X_val_no_icu)
X_test_np_no_icu  = pre_no_icu.transform(X_test_no_icu)

# 3️⃣ Torch conversion
Xtr_no_icu = torch.tensor(X_train_np_no_icu, dtype=torch.float32)
ytr_no_icu = torch.tensor(y_train.values.reshape(-1,1), dtype=torch.float32)
Xva_no_icu = torch.tensor(X_val_np_no_icu,   dtype=torch.float32)
yva_no_icu = torch.tensor(y_val.values.reshape(-1,1),   dtype=torch.float32)
Xte_no_icu = torch.tensor(X_test_np_no_icu,  dtype=torch.float32)
yte_no_icu = torch.tensor(y_test.values.reshape(-1,1),  dtype=torch.float32)

train_loader_no_icu = DataLoader(TensorDataset(Xtr_no_icu, ytr_no_icu), batch_size=64, shuffle=True)
val_loader_no_icu   = DataLoader(TensorDataset(Xva_no_icu, yva_no_icu), batch_size=256)
test_loader_no_icu  = DataLoader(TensorDataset(Xte_no_icu, yte_no_icu), batch_size=256)

# 4️⃣ New MLP
mlp_no_icu = MLP(Xtr_no_icu.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(mlp_no_icu.parameters(), lr=1e-3)

# 5️⃣ Training loop
best_val = np.inf
patience, bad = 8, 0
epochs = 50
for ep in range(1, epochs+1):
    mlp_no_icu.train(); train_loss = 0.0
    for xb, yb in train_loader_no_icu:
        optimizer.zero_grad()
        pred = mlp_no_icu(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader_no_icu.dataset)

    mlp_no_icu.eval(); val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader_no_icu:
            val_loss += criterion(mlp_no_icu(xb), yb).item() * xb.size(0)
    val_loss /= len(val_loader_no_icu.dataset)

    if val_loss + 1e-6 < best_val:
        best_val = val_loss; bad = 0
        best_state = {k: v.cpu().clone() for k,v in mlp_no_icu.state_dict().items()}
    else:
        bad += 1
        if bad >= patience:
            break

mlp_no_icu.load_state_dict(best_state)

# 6️⃣ Test performance
mlp_no_icu.eval()
with torch.no_grad():
    y_pred_no_icu = np.vstack([mlp_no_icu(xb).cpu().numpy() for xb, _ in test_loader_no_icu]).ravel()

mse_test_no_icu = mean_squared_error(y_test, y_pred_no_icu)
mae_test_no_icu = mean_absolute_error(y_test, y_pred_no_icu)
print(f"No ICU features → Test MSE: {mse_test_no_icu:.4f} | MAE: {mae_test_no_icu:.4f}")
